In [7]:
!pip install mediapipe

In [8]:
import cv2
import mediapipe as mp
import numpy as np
from math import atan2, degrees
from IPython.display import HTML
from base64 import b64encode

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils


def angle_2d(a, b):
    """Returns angle between horizontal and line a->b in degrees."""
    return degrees(atan2(b[1] - a[1], b[0] - a[0]))


def distance(a, b):
    """Euclidean distance between 2D points."""
    return np.hypot(a[0] - b[0], a[1] - b[1])


def evaluate_tripod_strict(landmarks, image_shape):
    """Compute if person is in tripod position using stricter geometric rules."""
    h, w, _ = image_shape

    def get_landmark(idx):
        lm = landmarks[idx]
        return np.array([lm.x * w, lm.y * h])

    # Points clés
    shoulder_r = get_landmark(mp_pose.PoseLandmark.RIGHT_SHOULDER.value)
    shoulder_l = get_landmark(mp_pose.PoseLandmark.LEFT_SHOULDER.value)
    hip_r = get_landmark(mp_pose.PoseLandmark.RIGHT_HIP.value)
    hip_l = get_landmark(mp_pose.PoseLandmark.LEFT_HIP.value)
    knee_r = get_landmark(mp_pose.PoseLandmark.RIGHT_KNEE.value)
    knee_l = get_landmark(mp_pose.PoseLandmark.LEFT_KNEE.value)
    wrist_r = get_landmark(mp_pose.PoseLandmark.RIGHT_WRIST.value)
    wrist_l = get_landmark(mp_pose.PoseLandmark.LEFT_WRIST.value)

    # Centres
    shoulder = (shoulder_r + shoulder_l) / 2
    hip = (hip_r + hip_l) / 2
    knee_mid = (knee_r + knee_l) / 2

    # Distance de référence : largeur des épaules
    ref_dist = distance(shoulder_r, shoulder_l)

    # --- 1️⃣ Vérifier que le torse est fortement penché en avant ---
    torso_angle = abs(angle_2d(hip, shoulder))  # 90 = vertical
    leaning_forward = (35 <= torso_angle <= 60)

    # --- 2️⃣ Vérifier que la personne est clairement assise ---
    hip_to_knee_ratio = (hip[1] - knee_mid[1]) / ref_dist
    # Hanches à hauteur des genoux (valeurs plus grandes = personne debout)
    sitting_like = (-0.5 < hip_to_knee_ratio < 0.2)

    # --- 3️⃣ Vérifier que les mains sont très proches des genoux ---
    dist_r = distance(wrist_r, knee_r) / ref_dist
    dist_l = distance(wrist_l, knee_l) / ref_dist
    hands_near_knees = (dist_r < 0.6 and dist_l < 0.6)

    # --- 4️⃣ Vérifier que les mains sont en avant du corps (pas derrière) ---
    hands_forward = ((wrist_r[1] > shoulder_r[1]) and (wrist_l[1] > shoulder_l[1]))

    # --- 5️⃣ Vérifier la symétrie gauche/droite ---
    symmetry = abs((wrist_r[1] - wrist_l[1])) < (0.3 * ref_dist)

    # --- 6️⃣ Combiner avec pondérations plus strictes ---
    score = (0.35 * leaning_forward +
             0.25 * sitting_like +
             0.25 * hands_near_knees +
             0.10 * hands_forward +
             0.05 * symmetry)

    is_tripod = (score > 0.75)

    return {
        "tripod": is_tripod,
        "score": score,
        "leaning_forward": leaning_forward,
        "sitting_like": sitting_like,
        "hands_near_knees": hands_near_knees,
        "hands_forward": hands_forward,
        "symmetry": symmetry,
        "torso_angle": torso_angle,
        "hip_to_knee_ratio": hip_to_knee_ratio,
        "dist_r": dist_r,
        "dist_l": dist_l
    }


def process_video_notebook(input_path, output_path="tripod_output.mp4", display_every=60):
    """Process video frame-by-frame and optionally display progress inside notebook."""
    pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False)
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {input_path}")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_idx = 0
    tripod_frames = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            res = evaluate_tripod_strict(results.pose_landmarks.landmark, frame.shape)

            label = f"Tripod: {'YES' if res['tripod'] else 'NO'}  score={res['score']:.2f}"
            color = (0, 255, 0) if res['tripod'] else (0, 0, 255)
            cv2.rectangle(frame, (5, 5), (400, 55), (0, 0, 0), -1)
            cv2.putText(frame, label, (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            if res['tripod']:
                tripod_frames += 1

        writer.write(frame)

        if frame_idx % display_every == 0:
            print(f"Processed frame {frame_idx}")

        frame_idx += 1

    cap.release()
    writer.release()
    pose.close()

    tripod_ratio = tripod_frames / max(1, frame_idx)
    print(f"✅ Done! Tripod detected in {tripod_ratio*100:.1f}% of frames.")
    print(f"Output saved to: {output_path}")

    # Display video inline in notebook
    mp4 = open(output_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'<video controls loop width="640"><source src="{data_url}" type="video/mp4"></video>')

In [10]:
input_video = "first_test.mp4"

# Run detection
output = process_video_notebook(input_video, output_path="tripod_detected.mp4")

Processed frame 0
Processed frame 60
Processed frame 120
Processed frame 180
Processed frame 240
Processed frame 300
Processed frame 360
Processed frame 420
Processed frame 480
Processed frame 540
Processed frame 600
Processed frame 660
Processed frame 720
Processed frame 780
Processed frame 840
✅ Done! Tripod detected in 0.0% of frames.
Output saved to: tripod_detected.mp4
